### Adding xgboost jars for pyspark

In [1]:

import os
os.environ["HADOOP_USER_NAME"] = "hdfs"
os.environ["PYTHON_VERSION"] = "3.5.2"
# os.environ["PYSPARK_SUBMIT_ARGS"]

In [2]:
web_app_model_path = "/Users/apple/Desktop/flask_projects/repayment-api/final_model"

In [3]:
import numpy as np
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer,OneHotEncoderEstimator,VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.types import DoubleType
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *



In [4]:

spark = SparkSession.builder \
    .config('spark.master', 'local[2]') \
    .appName('Final-Project') \
    .getOrCreate()

#     .config("spark.jars.packages","spark-avro_2.12-3.0.0-preview2")\

#  .config("spark.jars", "xgboost4j-0.90.jar,xgboost4j-spark-0.90.jar")\
#     .config("spark.driver.extraClassPath", "xgboost4j-0.90.jar,xgboost4j-spark-0.90.jar")\
#     .config("spark.executor.extraClassPath", "xgboost4j-0.90.jar,xgboost4j-spark-0.90.jar")\


sc = spark.sparkContext

### Loading from hdfs

In [17]:
db_cols = ["ID", "INSTNM", "HIGHDEG", "CONTROL", "ADM_RATE", "ADM_RATE_ALL", "SATMTMID", "ACTCMMID", "ACTENMID", "ACTMTMID", "ACTWRMID", "SAT_AVG", "SAT_AVG_ALL", "UGDS", "UGDS_WHITE", "UGDS_BLACK", "UGDS_HISP", "UGDS_ASIAN", "UGDS_AIAN", "UGDS_NHPI", "UGDS_2MOR", "UGDS_NRA", "UGDS_UNKN", "PPTUG_EF", "COSTT4_A", "COSTT4_P", "TUITIONFEE_IN", "TUITIONFEE_OUT", "TUITIONFEE_PROG", "TUITFTE", "INEXPFTE", "AVGFACSAL", "PCTPELL", "PCTFLOAN", "COMP_ORIG_YR2_RT", "DEATH_YR3_RT", "COMP_ORIG_YR3_RT", "LOAN_DEATH_YR3_RT", "LOAN_COMP_ORIG_YR3_RT", "DEATH_YR4_RT", "COMP_ORIG_YR4_RT", "COMPL_RPY_1YR_RT", "AGE_ENTRY", "COUNT_NWNE_P10", "COUNT_WNE_P10", "MN_EARN_WNE_P10", "MD_EARN_WNE_P10", "YEARS"]

In [22]:
# scorecard_data =  spark.read.format("csv")\
#   .option("header", "true")\
#   .option("inferSchema", "true")\
#   .load("../data/CollegeScorecard_Raw_Data/MERGED2012_13_PP.csv")\

scorecard_data_rdd =  spark.read.format("csv").load("hdfs://localhost:9000/repayment/scorecard.txt")
# spark.read\
#   .load("hdfs://localhost:9000/repayment/scorecard_data.csv")\




In [25]:
scorecard_data = scorecard_data_rdd.toDF(*db_cols)

In [26]:

scorecard_data.limit(1).toPandas()


,ID,INSTNM,HIGHDEG,CONTROL,ADM_RATE,ADM_RATE_ALL,SATMTMID,ACTCMMID,ACTENMID,ACTMTMID,...,LOAN_COMP_ORIG_YR3_RT,DEATH_YR4_RT,COMP_ORIG_YR4_RT,COMPL_RPY_1YR_RT,AGE_ENTRY,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10,YEARS
0,1,Alabama A & M University,4,1,0.49,0.49,None,18.0,18.0,17.0,...,0.113350125945,PrivacySuppressed,0.233532934132,PrivacySuppressed,null,null,null,null,null,2008


In [27]:
features =["CONTROL","ADM_RATE","ADM_RATE_ALL","SAT_AVG_ALL","SATMTMID","UGDS","HIGHDEG",  "TUITFTE", 
       "COSTT4_A", "PCTFLOAN","COMP_ORIG_YR2_RT", "UGDS_WHITE","UGDS_BLACK","UGDS_HISP","UGDS_ASIAN","UGDS_AIAN","UGDS_NHPI","UGDS_2MOR","UGDS_NRA","UGDS_UNKN","PPTUG_EF","COSTT4_P","TUITIONFEE_IN","TUITIONFEE_OUT","TUITIONFEE_PROG","INEXPFTE","PCTPELL","COMP_ORIG_YR3_RT","LOAN_COMP_ORIG_YR3_RT","DEATH_YR4_RT","COMP_ORIG_YR4_RT","AGE_ENTRY","COUNT_NWNE_P10","COUNT_WNE_P10","MN_EARN_WNE_P10","MD_EARN_WNE_P10","COMPL_RPY_1YR_RT"]


In [28]:
categ_features = ["CONTROL","HIGHDEG"]
target = "COMPL_RPY_1YR_RT"

In [29]:
# this preprocessing step is important only in training, because in run time we will not receive any value as privacySuppressed, but it will be null
def cleanPrivacySuppressed(dataFrame):
    # define udf to clean PrivacySuppressed for dataframe  column
    clean_privacy_suppressed =F.udf(lambda value: np.nan if value == "PrivacySuppressed" else value)
    # apply the function on every column in dataframe
    dataFrame = dataFrame.select([ clean_privacy_suppressed(c).cast("double" if c not in categ_features else "int")  for c in dataFrame.columns if c in features])
    # restore old names of columns (previously applied function changed col names)
    for index in range(len(dataFrame.columns)):
        dataFrame = dataFrame.withColumnRenamed(dataFrame.columns[index], dataFrame.columns[index].replace("CAST(<lambda>(","").replace(") AS DOUBLE)","").replace(") AS INT)",""))     
    return dataFrame
    
    

In [30]:
# this preprocessing step is important only in training, so it will not be included in the pipeline.
scorecard_data_cleaned = cleanPrivacySuppressed(scorecard_data)
# select not null target rows
scorecard_data_cleaned =scorecard_data_cleaned.where(F.col(target).isNotNull()).where(F.col(target)!= np.nan)

In [31]:
# hot encoder preprocessing
hot_encoder = OneHotEncoderEstimator(inputCols=categ_features, outputCols=["{0}_ENCODED".format(colName) for colName in categ_features] )


In [32]:
#vector assembler
model_input_features = [f for f in features if f not in categ_features and f is not target ]
model_input_features.extend(["{0}_ENCODED".format(colName) for colName in categ_features])
vec_assembler = VectorAssembler(inputCols=model_input_features,outputCol="features",handleInvalid="keep")



In [33]:
#preprocessing pipleline
preprocessing_pipeline = Pipeline(stages=[hot_encoder,vec_assembler])
preprocessed_data = preprocessing_pipeline.fit(scorecard_data_cleaned).transform(scorecard_data_cleaned)
# cache this preprocessing step, this is performance optimization step for model-tunning process
preprocessed_data.cache()

DataFrame[HIGHDEG: int, CONTROL: int, ADM_RATE: double, ADM_RATE_ALL: double, SATMTMID: double, SAT_AVG_ALL: double, UGDS: double, UGDS_WHITE: double, UGDS_BLACK: double, UGDS_HISP: double, UGDS_ASIAN: double, UGDS_AIAN: double, UGDS_NHPI: double, UGDS_2MOR: double, UGDS_NRA: double, UGDS_UNKN: double, PPTUG_EF: double, COSTT4_A: double, COSTT4_P: double, TUITIONFEE_IN: double, TUITIONFEE_OUT: double, TUITIONFEE_PROG: double, TUITFTE: double, INEXPFTE: double, PCTPELL: double, PCTFLOAN: double, COMP_ORIG_YR2_RT: double, COMP_ORIG_YR3_RT: double, LOAN_COMP_ORIG_YR3_RT: double, DEATH_YR4_RT: double, COMP_ORIG_YR4_RT: double, COMPL_RPY_1YR_RT: double, AGE_ENTRY: double, COUNT_NWNE_P10: double, COUNT_WNE_P10: double, MN_EARN_WNE_P10: double, MD_EARN_WNE_P10: double, CONTROL_ENCODED: vector, HIGHDEG_ENCODED: vector, features: vector]

In [34]:
# split data to train/test 80/20
train_preprocessed_data = preprocessed_data.randomSplit([.8,.2])[0]
train_preprocessed_data.cache()

DataFrame[HIGHDEG: int, CONTROL: int, ADM_RATE: double, ADM_RATE_ALL: double, SATMTMID: double, SAT_AVG_ALL: double, UGDS: double, UGDS_WHITE: double, UGDS_BLACK: double, UGDS_HISP: double, UGDS_ASIAN: double, UGDS_AIAN: double, UGDS_NHPI: double, UGDS_2MOR: double, UGDS_NRA: double, UGDS_UNKN: double, PPTUG_EF: double, COSTT4_A: double, COSTT4_P: double, TUITIONFEE_IN: double, TUITIONFEE_OUT: double, TUITIONFEE_PROG: double, TUITFTE: double, INEXPFTE: double, PCTPELL: double, PCTFLOAN: double, COMP_ORIG_YR2_RT: double, COMP_ORIG_YR3_RT: double, LOAN_COMP_ORIG_YR3_RT: double, DEATH_YR4_RT: double, COMP_ORIG_YR4_RT: double, COMPL_RPY_1YR_RT: double, AGE_ENTRY: double, COUNT_NWNE_P10: double, COUNT_WNE_P10: double, MN_EARN_WNE_P10: double, MD_EARN_WNE_P10: double, CONTROL_ENCODED: vector, HIGHDEG_ENCODED: vector, features: vector]

In [35]:
#model
gbmodel = GBTRegressor(featuresCol="features",labelCol=target)

In [36]:
# model tuning process
evaluator =RegressionEvaluator(labelCol=target)
paramGrid = (ParamGridBuilder()
             .addGrid(gbmodel.maxDepth, [2, 4, 6])
             .addGrid(gbmodel.maxBins, [20, 60])
             .addGrid(gbmodel.maxIter, [10, 20])
             .addGrid(gbmodel.minInfoGain, [0.0, 0.05])
             .build())
cv = CrossValidator(estimator=gbmodel, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)


pipeline_cv = cv.fit(train_preprocessed_data)



In [16]:
# final pipeline to deploy is the preprocessing steps + best model (best hyperparameters)
final_pipeline = Pipeline(stages=[*preprocessing_pipeline.getStages(), pipeline_cv.bestModel])

In [17]:
#train on all data and save model to disk
final_model = final_pipeline.fit(scorecard_data_cleaned)
final_model.write().overwrite().save(web_app_model_path)